In [ ]:
!pip install torch torchtext transformers sentencepiece pandas tqdm datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from datasets import load_dataset, DatasetDict, Dataset
import pandas as pd
import ast
import datasets
from tqdm import tqdm
import time

In [ ]:
#load data
data_sample = load_dataset("QuyenAnhDE/Diseases_Symptoms")

Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/400 [00:00<?, ? examples/s]

In [ ]:
data_sample

DatasetDict({
    train: Dataset({
        features: ['Code', 'Name', 'Symptoms', 'Treatments'],
        num_rows: 400
    })
})

In [ ]:
updated_data = [{'Name': item['Name'],'Symptoms': item['Symptoms']} for item in data_sample['train']]

In [ ]:
df = pd.DataFrame(updated_data)

In [ ]:
df.head(5)

,Name,Symptoms
0,Panic disorder,"Palpitations, Sweating, Trembling, Shortness o..."
1,Vocal cord polyp,"Hoarseness, Vocal Changes, Vocal Fatigue"
2,Turner syndrome,"Short stature, Gonadal dysgenesis, Webbed neck..."
3,Cryptorchidism,"Absence or undescended testicle(s), empty scro..."
4,Ethylene glycol poisoning-1,"Nausea, vomiting, abdominal pain, General mala..."


In [ ]:
# just extracting symptoms
df['Symptoms'] = df['Symptoms'].apply(lambda x: ','.join(x.split(', ')))

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda')

else:
  try:
   device = torch.device('mps')
  except Exception:
   device = torch.device('cpu')

In [ ]:
device = torch.device("mps") if torch.backends.mps.is_built() else torch.device("cpu")

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('distilbert/distilgpt2')
model = GPT2LMHeadModel.from_pretrained('distilbert/distilgpt2').to(device)

In [ ]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
BATCH_SIZE = 8

In [ ]:
df.describe()

,Name,Symptoms
count,400,400
unique,392,395
top,Sciatica,"Swelling,pain,dry mouth,bad taste"
freq,3,3


In [ ]:
# Dataset Prep
class LanguageDataset(Dataset):
    """
    An extension of the Dataset object to:
      - Make training loop cleaner
      - Make ingestion easier from pandas df's
    """
    def __init__(self, df, tokenizer):
        self.labels = df.columns
        self.data = df.to_dict(orient='records')
        self.tokenizer = tokenizer
        x = self.fittest_max_length(df)  # Fix here
        self.max_length = x

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx][self.labels[0]]
        y = self.data[idx][self.labels[1]]
        text = f"{x} | {y}"
        tokens = self.tokenizer.encode_plus(text, return_tensors='pt', max_length=128, padding='max_length', truncation=True)
        return tokens

    def fittest_max_length(self, df):  # Fix here
        """
        Smallest power of two larger than the longest term in the data set.
        Important to set up max length to speed training time.
        """
        max_length = max(len(max(df[self.labels[0]], key=len)), len(max(df[self.labels[1]], key=len)))
        x = 2
        while x < max_length: x = x * 2
        return x



In [ ]:
data_sample = LanguageDataset(df, tokenizer)

In [ ]:
data_sample

In [ ]:
#creating train size
train_size = int(0.8 * len(data_sample))
valid_size = len(data_sample) - train_size
train_data, valid_data = random_split(data_sample, [train_size, valid_size])

In [ ]:
# Making the iterators
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=BATCH_SIZE)

In [ ]:
#setting no of epochs
num_epochs = 8

In [ ]:
#Training parameters
batch_size = BATCH_SIZE
model_name = 'distilgpt2'
gpu = 0

In [ ]:
# Seting the learning rate and loss function
## CrossEntropyLoss measures how close answers to the truth.
## More punishing for high confidence wrong answers
criterion = nn.CrossEntropyLoss(ignore_index = tokenizer.pad_token_id)
optimizer = optim.Adam(model.parameters(), lr=5e-4)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# Init a results dataframe
results = pd.DataFrame(columns=['epoch', 'transformer', 'batch_size', 'gpu',
                                'training_loss', 'validation_loss', 'epoch_duration_sec'])

In [ ]:
# The training loop
for epoch in range(num_epochs):
    start_time = time.time()  # Start the timer for the epoch

    # Training
    ## This line tells the model we're in 'learning mode'
    model.train()
    epoch_training_loss = 0
    train_iterator = tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs} Batch Size: {batch_size}, Transformer: {model_name}")
    for batch in train_iterator:
        optimizer.zero_grad()
        inputs = batch['input_ids'].squeeze(1).to(device)
        targets = inputs.clone()
        outputs = model(input_ids=inputs, labels=targets)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        train_iterator.set_postfix({'Training Loss': loss.item()})
        epoch_training_loss += loss.item()
    avg_epoch_training_loss = epoch_training_loss / len(train_iterator)

    # Validation
    ## This line below tells the model to 'stop learning'
    model.eval()
    epoch_validation_loss = 0
    total_loss = 0
    valid_iterator = tqdm(valid_loader, desc=f"Validation Epoch {epoch+1}/{num_epochs}")
    with torch.no_grad():
        for batch in valid_iterator:
            inputs = batch['input_ids'].squeeze(1).to(device)
            targets = inputs.clone()
            outputs = model(input_ids=inputs, labels=targets)
            loss = outputs.loss
            total_loss += loss
            valid_iterator.set_postfix({'Validation Loss': loss.item()})
            epoch_validation_loss += loss.item()

    avg_epoch_validation_loss = epoch_validation_loss / len(valid_loader)

    end_time = time.time()  # End the timer for the epoch
    epoch_duration_sec = end_time - start_time  # Calculate the duration in seconds

    new_row = {'transformer': model_name,
               'batch_size': batch_size,
               'gpu': gpu,
               'epoch': epoch+1,
               'training_loss': avg_epoch_training_loss,
               'validation_loss': avg_epoch_validation_loss,
               'epoch_duration_sec': epoch_duration_sec}  # Add epoch_duration to the dataframe

    results.loc[len(results)] = new_row
    print(f"Epoch: {epoch+1}, Validation Loss: {total_loss/len(valid_loader)}")

Training Epoch 1/8 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [07:19<00:00, 10.99s/it, Training Loss=0.921]
Validation Epoch 1/8: 100%|██████████| 10/10 [00:35<00:00,  3.50s/it, Validation Loss=0.761]


Epoch: 1, Validation Loss: 0.6054503321647644


Training Epoch 2/8 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [07:09<00:00, 10.73s/it, Training Loss=0.609]
Validation Epoch 2/8: 100%|██████████| 10/10 [00:36<00:00,  3.69s/it, Validation Loss=0.745]


Epoch: 2, Validation Loss: 0.5807095170021057


Training Epoch 3/8 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [07:09<00:00, 10.74s/it, Training Loss=0.428]
Validation Epoch 3/8: 100%|██████████| 10/10 [00:33<00:00,  3.31s/it, Validation Loss=0.73]


...

Epoch: 6, Validation Loss: 0.6909502744674683


Training Epoch 7/8 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [07:07<00:00, 10.68s/it, Training Loss=0.193]
Validation Epoch 7/8: 100%|██████████| 10/10 [00:33<00:00,  3.32s/it, Validation Loss=0.934]


Epoch: 7, Validation Loss: 0.7338318824768066


Training Epoch 8/8 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [07:08<00:00, 10.70s/it, Training Loss=0.118]
Validation Epoch 8/8: 100%|██████████| 10/10 [00:34<00:00,  3.42s/it, Validation Loss=0.95]

Epoch: 8, Validation Loss: 0.7784565091133118


In [68]:
input_str = "Kidney Failure"



In [69]:
input_ids = tokenizer.encode(input_str, return_tensors='pt').to(device)

In [70]:
input_ids

tensor([[48374,  1681, 25743]])

In [71]:
output = model.generate(
    input_ids,
    max_length=20,
    num_return_sequences=1,
    do_sample=True,
    top_k=8,
    top_p=0.95,
    temperature=0.5,
    repetition_penalty=1.2
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [72]:
output

tensor([[48374,  1681, 25743,   930,  2275,  3438,  1292,  2356,    11, 18041,
           287,   262, 18922,    11,    69, 46018,  2956,  1883, 50256]])

In [73]:
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

In [74]:
decoded_output

'Kidney Failure | Abdominal pain,blood in the urine,frequent urination'

In [ ]:
torch.save(model, 'MedSymptomGPT.pt')

In [ ]:
torch.save(model, 'drive/My Drive/MedSymptomGPT.pt')
